# Prediction task is to determine whether a person's income is over $50,000 a year.

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, auc, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTE

In [ ]:
data = pd.read_csv('/kaggle/input/adult-income/adult_income.csv')

data.head(20)

In [ ]:
data['income'].unique()

In [ ]:
data['income'].value_counts()

Target Feature is **income column** and independent variables are the others.<br>
This is **binary classification. Label has only two kinds.**

In [ ]:
data.shape

### we have 48842 rows and 15 columns. - before cleaning

In [ ]:
data.dtypes

In [ ]:
#Find missing values
data.isnull().sum()

we have missing values of "workclass", "occupation" and "native_country". 

In [ ]:
data.duplicated().sum()

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.duplicated().sum()

In [ ]:
data.shape

In [ ]:
data['workclass'].unique()

In [ ]:
data['occupation'].unique()

In [ ]:
data['workclass'] = data.groupby(['education','age'])['workclass'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)

In [ ]:
data.isnull().sum()

In [ ]:
data[data['workclass'].isna()]

I can't find any special things so I decided to fill new category which is 'unknown'.

In [ ]:
data['workclass'].fillna('Unknown', inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data['occupation'] = data.groupby(['workclass','education'])['occupation'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)

In [ ]:
data.isnull().sum()

In [ ]:
data[data['occupation'].isna()]

In [ ]:
data['occupation'] = data.groupby('education')['occupation'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)

In [ ]:
data.isnull().sum()

In [ ]:
data['native_country'] = data.groupby('race')['native_country'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)

In [ ]:
data.isnull().sum()

In [ ]:
edu_check = data.groupby("education")["education_num"].unique()
edu_check.sort_values(ascending=True)

In [ ]:
edu_check = data.groupby("education")["education_num"].nunique()
print(edu_check[edu_check > 1])

education_num is good matching with education column.<br>
It is continuous and the values are consistent.

### Finding Outliers

In [ ]:
data['income'].value_counts()

After checking modeling performance, I will decide whether use SMOTE or not.

In [ ]:
data.describe()

In [ ]:
data[data['capital_gain']== 99999]['age'].sum()

In capital_gain column, the mean(1080) is much smaller than the standard deviation(7455), and all quartiles (25, 50, 75%) are 0.<br> The maximum value is likely to be an outlier.

In [ ]:
captial_median = data['capital_gain'].median()
data.loc[data['capital_gain']== 99999, 'capital_gain'] = captial_median

In [ ]:
data['capital_gain'].unique()

In [ ]:
sns.boxplot(data=data)

In [ ]:
data[(data['hours_per_week']>70) & (data['age'] > 60)]['age'].sum()

that doesn't make sense so I replaced median instead of that range.

In [ ]:
hours_median = data['hours_per_week'].median()
data.loc[(data['age'] > 60) & (data['hours_per_week'] > 70), 'hours_per_week'] = hours_median

In [ ]:
# only numerical data type
columns = data.select_dtypes(include=['number']).columns

for value in columns:
    Q1 = data[value].quantile(0.25)
    Q3 = data[value].quantile(0.75)
    IQR = Q3 - Q1 
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # check number of outliers
    outliers = data[(data[value] < lower_bound) | (data[value] > upper_bound)]
    num_outliers = outliers.shape[0]
    outlier_ratio = (num_outliers / len(data[value])) * 100
    
    print(f"{value}: {outlier_ratio: .2f}%")

In [ ]:
# fig : total, axes : each graph, 1 means one row
fig, axes = plt.subplots(1, len(columns), figsize=(15,5))

fig.suptitle('Plot the histograms for numerical data')

# create each histogram 
for i, col in enumerate(columns):
    sns.histplot(data[col], bins=10, kde=True, ax=axes[i])
    axes[i].set_title(f'Histogram of {col}') #subtitle

plt.tight_layout()
plt.show()

In [ ]:
# dealing with outliers of 'hours per week' column
Q1 = data['hours_per_week'].quantile(0.25)
Q3 = data['hours_per_week'].quantile(0.75)
IQR = Q3 - Q1 

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# set median instead of outlier
median_value = data['hours_per_week'].median()
data.loc[(data['hours_per_week'] < lower_bound) | (data['hours_per_week'] > upper_bound), 'hours_per_week'] = median_value

The distribution of capital_gain is too skewed, so replacing outliers with the median might reduce data reliability.<br>
So I found another way to deal with outliers. <br>

In [ ]:
# add new category (binary) for Trees model
data['has_capital_gain'] = (data['capital_gain'] > 0).astype(int)
# change log for reducing scale for linear model
data['capital_gain_log'] = np.log1p(data['capital_gain'])  # log(x+1)

In [ ]:
columns = data.select_dtypes(include=['number']).columns

# fig : total, axes : each graph, 1 means one row
fig, axes = plt.subplots(1, len(columns), figsize=(15,5))

fig.suptitle('Plot the histograms for numerical data')

# create each histogram 
for i, col in enumerate(columns):
    sns.histplot(data[col], bins=10, kde=True, ax=axes[i])
    axes[i].set_title(f'Histogram of {col}') #subtitle

plt.tight_layout()
plt.show()

In [ ]:
for value in columns:
    Q1 = data[value].quantile(0.25)
    Q3 = data[value].quantile(0.75)
    IQR = Q3 - Q1 
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # check number of outliers
    outliers = data[(data[value] < lower_bound) | (data[value] > upper_bound)]
    num_outliers = outliers.shape[0]
    outlier_ratio = (num_outliers / len(data[value])) * 100
    
    print(f"{value}: {outlier_ratio: .2f}%")

I reduced around **2%** outliers of 'hours_per_week' coulmn. the left outliers will be scaled using feature scaling.

In [ ]:
data.describe()

In [ ]:
data.duplicated().sum()

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.duplicated().sum()

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

### after cleaning, we have 48735 rows and 17 columns

### Category Data type to Numerical Data type

In [ ]:
data.info()

In [ ]:
# for education, it already has numerical column which is education_num.
cat_col = ['workclass','marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'income']

In [ ]:
for col in cat_col:
    print(f"{col} : {data[col].nunique()}")

In [ ]:
la_encoder = LabelEncoder()
data['workclass_en'] = la_encoder.fit_transform(data['workclass'])
data['marital_status_en'] = la_encoder.fit_transform(data['marital_status'])
data['occupation_en'] = la_encoder.fit_transform(data['occupation'])
data['relationship_en'] = la_encoder.fit_transform(data['relationship'])
data['native_country_en'] = la_encoder.fit_transform(data['native_country'])
data['race_trees'] = la_encoder.fit_transform(data['race'])

In [ ]:
# one-hot encoding
data = pd.get_dummies(data, columns=['race'], drop_first=True)

In [ ]:
data.head()

In [ ]:
data['sex'].value_counts(dropna=False)

for "sex" and "income" column, I will recreate new binaries columns.

In [ ]:
data['sex'] = data['sex'].map({'Male': 0, 'Female': 1})

In [ ]:
data['income'].value_counts(dropna=False)

In [ ]:
data['income'] = data['income'].map({'<=50K': 0, '>50K': 1})

In [ ]:
data.head()

In [ ]:
data['income'].head()

## visualize the correlation matrix.

when I perform correlation, we need to pick the column which is encoded.

In [ ]:
data.columns

feature scaling is not neseccery for **trees model**.

In [ ]:
tree_data = data[['age', 'fnlwgt', 'education_num', 'sex',
                        'has_capital_gain', 'capital_loss', 'hours_per_week',
                        'workclass_en', 'marital_status_en', 'occupation_en',
                        'relationship_en', 'native_country_en', 'race_trees', 'income' ]]
tree_data.columns

In [ ]:
numeric_data = data[['age', 'fnlwgt', 'education_num', 'sex',
                        'capital_gain_log', 'capital_loss', 'hours_per_week',
                        'workclass_en', 'marital_status_en', 'occupation_en',
                        'relationship_en', 'native_country_en', 'race_Asian-Pac-Islander', 'race_Black', 'race_Other', 'race_White', 'income' ]]
corr = numeric_data.corr()
corr

In [ ]:
#Correlation matrix
plt.figure(figsize = (16,16))
sns.heatmap(corr, annot=True, cmap='Reds', fmt='.2f')

### the top 3 features that are highly correlated with the target feature

In [ ]:
# correlation with target feature('income')
# top3
top_features = corr['income'].abs().sort_values(ascending=False).iloc[1:4] 
top_features

For linear model data set, **"education_num", "capital_gain_log", "age"** columns are highly correlated with target features.

In [ ]:
corr_tree = tree_data.corr()
corr_tree

In [ ]:
#Correlation matrix
plt.figure(figsize = (16,16))
sns.heatmap(corr_tree, annot=True, cmap='Reds', fmt='.2f')

In [ ]:
# correlation with target feature('income')
# top3
top_features = corr_tree['income'].abs().sort_values(ascending=False).iloc[1:4] 
top_features

For Trees model data set, **"education_num", "relationshop_en", "capital_gain"** columns are highly correlated with target features.

## Feature Scaling

In [ ]:
X = numeric_data.drop(['income'], axis=1)
Y = numeric_data['income']

In [ ]:
X_co = X
Y_co = Y

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

I have a lot of outliers so, I decided to use robust scaler.

In [ ]:
ro_scaler = RobustScaler()
X_train_scaled = ro_scaler.fit_transform(X_train)
X_test_scaled = ro_scaler.transform(X_test)

In [ ]:
X_train = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test = pd.DataFrame(X_test_scaled, columns=X.columns)

In [ ]:
X_train

## ML modeling

### Logistic Regression

In [ ]:
# Logistic Regression
logreg = LogisticRegression(random_state=16, max_iter=1000)
logreg.fit(X_train, Y_train)

# probablity for auc-roc cure
lg_probs = logreg.predict_proba(X_test)
lg_probs

In [ ]:
y_pred_train = logreg.predict(X_train)
y_pred = logreg.predict(X_test)

conf_matrix = confusion_matrix(Y_test, y_pred)
conf_matrix

- True Negatives (TN) = 6952<br>
The model correctly predicted 6952 people as earning ≤50K.

- False Positives (FP) = 461<br>
The model incorrectly predicted 461 people as earning >50K when they actually earn ≤50K (Type I error).

- False Negatives (FN) = 1295<br>
The model incorrectly predicted 1295 people as earning ≤50K when they actually earn >50K (Type II error).

- True Positives (TP) = 1039<br>
The model correctly predicted 1039 people as earning >50K.

In [ ]:
# probablity

print(f"Train Accuracy score for Logistic Regression: {accuracy_score(Y_train, y_pred_train):.2f}")
print(f"Test Accuracy score for Logistic Regression: {accuracy_score(Y_test, y_pred):.2f}")


target_names = ['<=50K', '>50K']
print(classification_report(Y_test, y_pred, target_names = target_names))

- Key Observation
    1. Major issue: The model struggles to identify >50K earners, missing 55% of them due to the low recall of 0.45.
- Final Verdict
    1. Good accuracy (82%) with no overfitting.
    2. Severe class imbalance issue: The model is biased toward ≤50K and struggles with >50K earners (low recall of 0.45).
    3. High False Negatives for >50K → Many high-income individuals are misclassified as low-income.

### KNN

In [ ]:
error_rates = []
k_values = range(1, 31)
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, Y_train)
    Y_knn_pred = knn.predict(X_test)
    error = 1 - accuracy_score(Y_test, Y_knn_pred)
    error_rates.append(error)

In [ ]:
#Plot the elbow curve to find the optimal value of k
plt.figure(figsize=(20, 5))
plt.plot(k_values, error_rates, marker='o', linestyle='dashed', color='b', markersize=8)
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Error Rate')
plt.title('Elbow Method to Find Optimal k')
plt.xticks(np.arange(1, 31))
plt.grid(True)
plt.show()

**The error rate is the least for k = 26.**

In [ ]:
#Performing KNN with optimal value of k
knn = KNeighborsClassifier(n_neighbors=26)
knn.fit(X_train, Y_train)

Y_knn_probs = knn.predict_proba(X_test)
Y_knn_probs

y_pred_train = knn.predict(X_train)
Y_knn_pred = knn.predict(X_test)

conf_matrix = confusion_matrix(Y_test, Y_knn_pred)
conf_matrix

-	True Negatives (TN) = 6922<br>
The model correctly predicted 6922 people as earning ≤50K.
-	False Positives (FP) = 491<br>
The model incorrectly predicted 491 people as earning >50K, when they actually earn ≤50K (Type I error).
-	False Negatives (FN) = 1020<br>
The model incorrectly predicted 1020 people as earning ≤50K, when they actually earn >50K (Type II error).
-	True Positives (TP) = 1314<br>
The model correctly predicted 1314 people as earning >50K.


In [ ]:
print(f"Train Accuracy score for KNN: {accuracy_score(Y_train, y_pred_train):.2f}")
print(f"Test Accuracy score for KNN: {accuracy_score(Y_test, Y_knn_pred):.2f}")

target_names = ['<=50K', '>50K']
print(classification_report(Y_test, Y_knn_pred, target_names = target_names))

- Overall Observation
    1. The model struggles with >50K earners, misclassifying 44% of them as ≤50K.
- Final Verdict
    1. Good overall accuracy (84%) and strong recall for ≤50K (93%).
    2. Class imbalance issue: The model struggles with the >50K class, identifying only 56% of them correctly.
    3. High False Negatives for >50K: Many high earners are misclassified as ≤50K.

### Decision Tree

In [ ]:
# split data for trees model
X_t = tree_data.drop(['income'], axis=1)
Y_t = tree_data['income']

X_t_train, X_t_test, Y_t_train, Y_t_test = train_test_split(X_t, Y_t, test_size=0.2, random_state=42)

In [ ]:
#Find the best value for max_depth in decision tree
scores = []
depths = range(1, 21)
for d in depths:
    score = cross_val_score(DecisionTreeClassifier(criterion= 'entropy', max_depth=d), X_t_train, Y_t_train, cv=5)
    avg_score = mean(score)
    scores.append(avg_score)

best_depth = depths[np.argmax(scores)]
print(f"Best max_depth: {best_depth}")

In [ ]:
dt_clr_opt = DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=7)
dt_clr_opt.fit(X_t_train, Y_t_train)

#predict
y_pred_train = dt_clr_opt.predict(X_t_train)
y_pred_test = dt_clr_opt.predict(X_t_test)

conf_matrix = confusion_matrix(Y_t_test, y_pred_test)
conf_matrix

- True Negatives (TN) = 6914<br>
The model correctly predicted 6914 people as earning ≤50K.
- False Positives (FP) = 499<br>
The model incorrectly predicted 499 people as earning >50K, when they actually earn ≤50K (Type I error).
- False Negatives (FN) = 1096<br>
The model incorrectly predicted 1096 people as earning ≤50K, when  they actually earn >50K (Type II error).
- True Positives (TP) = 1238<br>
The model correctly predicted 1238 people as earning >50K.

In [ ]:
print(f"Train Accuracy:, {accuracy_score(Y_t_train, y_pred_train):.2f}")
print(f"Test Accuracy:, {accuracy_score(Y_t_test, y_pred_test):.2f}")
target_names = ['<=50K', '>50K']
print("\nClassification Report:\n", classification_report(Y_t_test, y_pred_test, target_names=target_names))

- Overall Observation
    1. The model struggles to correctly identify all >50K earners, misclassifying 47% of them as ≤50K.
- Final Verdict
    1. Good overall accuracy (84%) and strong recall for ≤50K (93%).
    2. Imbalance problem: The model struggles with the >50K class, only identifying 53% of them correctly.
    3. False Negatives for >50K: Many high earners are misclassified as ≤50K.

### Plot the decision tree

In [ ]:
from sklearn.tree import export_graphviz
from six import StringIO  
from IPython.display import Image  
import pydotplus
import os
os.environ["PATH"] += os.pathsep + "/opt/homebrew/bin/"

In [ ]:
# Export the decision tree to DOT format
dot_data = StringIO()

export_graphviz(
    dt_clr_opt,
    out_file=dot_data, 
    filled=True,  
    rounded=True,  
    special_characters=True,  
    feature_names=X_t_train.columns, 
    class_names=['Low income','High income']
)

# Generate graph
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png("decision_tree.png")  # Save as PNG

# Display the tree
Image(filename="decision_tree.png")

### SVM

In [ ]:
# to check what kernel should I use
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train)  # dimensionality reduction

plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=Y_train, palette='coolwarm', alpha=0.7)
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.title("PCA Projection of Data")
plt.legend(title="Class")
plt.show()

I couldn't find linear separation so I would use rbf kernel.

In [ ]:
svm = SVC() #Model building

param_grid = {"C": [1, 10, 100, 200, 300]}

# GridSearchCV
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring="accuracy", n_jobs=-1)
grid_search.fit(X_train, Y_train)

print(f"Best C: {grid_search.best_params_['C']}")

In [ ]:
svm_300 = SVC(random_state=42, C=300, probability=True)
svm_300.fit(X_train, Y_train)

Y_svm_probs = svm_300.predict_proba(X_test)

y_pred_train = svm_300.predict(X_train)
y_pred_test = svm_300.predict(X_test)

conf_matrix = confusion_matrix(Y_test, y_pred_test)
conf_matrix

-	True Negatives (TN) = 7072 <br>
The model correctly predicted 7072 people as earning ≤50K.
-	False Positives (FP) = 341<br>
The model incorrectly predicted 341 people as earning >50K, when they actually earn ≤50K (Type I error).
-	False Negatives (FN) = 1699<br>
The model incorrectly predicted 1699 people as earning ≤50K, when they actually earn >50K (Type II error).
-	True Positives (TP) = 635<br>
The model correctly predicted 635 people as earning >50K.

In [ ]:
# train and test accuracy
print(f"Train Accuracy:, {accuracy_score(Y_train, y_pred_train):.2f}")
print(f"Test Accuracy:, {accuracy_score(Y_test, y_pred_test):.2f}")

target_names = ['<=50K', '>50K']
print(classification_report(Y_test, y_pred_test, target_names = target_names))


- Overall Observation
    1. The model fails to identify 73% of actual >50K earners, which is a serious class imbalance problem.
- Final Verdict
    1. Moderate overall accuracy (79%), but extremely low recall for >50K (27%), meaning most high earners are not correctly identified.
    2. Severe class imbalance issue: The model performs well for ≤50K (95% recall) but fails to generalize for >50K.
    3. Extremely high False Negatives for >50K: A large proportion of high-income individuals are misclassified as ≤50K, making this model unreliable for detecting high earners.

### Random Forest

In [ ]:
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)

param_grid = {
    "n_estimators": [50, 100, 200],  # trees
    "max_depth": [10, 20, None],  
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4] 
}
# for better one
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring="accuracy", n_jobs=-1, verbose=2)
grid_search.fit(X_t_train, Y_t_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

In [ ]:
# the best model
best_rf = grid_search.best_estimator_

y_pred_train = best_rf.predict(X_t_train)
y_pred_test = best_rf.predict(X_t_test)

print(f"Train Accuracy: {accuracy_score(Y_t_train, y_pred_train):.2f}")
print(f"Test Accuracy: {accuracy_score(Y_t_test, y_pred_test):.2f}")
print()
# Confusion Matrix
conf_matrix = confusion_matrix(Y_t_test, y_pred_test)
print("Confusion Matrix:\n", conf_matrix)
print()
# Classification Report
print("Classification Report:\n", classification_report(Y_t_test, y_pred_test, target_names=["≤50K", ">50K"]))

- True Negatives (TN) = 6926<br>
The model correctly predicted 6926 people as earning ≤50K.
- False Positives (FP) = 487<br>
The model incorrectly predicted 487 people as earning >50K, when they actually earn ≤50K (Type I error).
- False Negatives (FN) = 956<br>
The model incorrectly predicted 956 people as earning ≤50K, when they actually earn >50K (Type II error).
- True Positives (TP) = 1378<br>
The model correctly predicted 1378 people as earning >50K.

- Overall Observations
    1. The model fails to correctly identify 41% of actual >50K earners, which can be problematic in applications where high-income classification is crucial.
    2. High FN count for >50K: Many high earners are misclassified, suggesting class imbalance issues.
- Final Verdict
    1. Decent accuracy (85%) and high recall for ≤50K earners (93%).
    2.  Slight overfitting (train accuracy 91% vs test accuracy 85%).
    3.  Moderate class imbalance issue: The model still underperforms for the >50K class (low recall of 59%).
    4.  Many False Negatives for >50K: The model fails to identify 41% of actual high earners, which reduces its reliability for classifying >50K incomes.

## AUC-ROC curve

In [ ]:
# Graph for only positive
Y_lr_probs = lg_probs[:, 1]
Y_knn_probs = Y_knn_probs[:, 1]
Y_dt_probs = dt_clr_opt.predict_proba(X_t_test)[:, 1]
Y_svm_probs = Y_svm_probs[:, 1]
Y_rf_probs = best_rf.predict_proba(X_t_test)[:, 1]

# Compute ROC curve and AUC for all the models
# Logistic Regression
fpr_lr, tpr_lr, _ = roc_curve(Y_test, Y_lr_probs)
auc_lr = auc(fpr_lr, tpr_lr)
# KNN
fpr_knn, tpr_knn, _ = roc_curve(Y_test, Y_knn_probs)
auc_knn = auc(fpr_knn, tpr_knn)
# Decision Trees
fpr_dt, tpr_dt, _ = roc_curve(Y_t_test, Y_dt_probs)
auc_dt = auc(fpr_dt, tpr_dt)
# SVM
fpr_svm, tpr_svm, _ = roc_curve(Y_test, Y_svm_probs)
auc_svm = auc(fpr_svm, tpr_svm)
# Random Forest
fpr_rf, tpr_rf, _ = roc_curve(Y_t_test, Y_rf_probs)
auc_rf = auc(fpr_rf, tpr_rf)

# Plot both ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr_lr, tpr_lr, color='blue', label=f'Logistic Regression (AUC = {auc_lr:.2f})')
plt.plot(fpr_knn, tpr_knn, color='red', linestyle='dashed', label=f'KNN (k=12) (AUC = {auc_knn:.2f})')
plt.plot(fpr_dt, tpr_dt, color='green', linestyle='dashdot', label=f'Decision Tree (AUC = {auc_dt:.2f})')
plt.plot(fpr_svm, tpr_svm, color='orange', linestyle='dashed', label=f'SVM (AUC = {auc_svm:.2f})')
plt.plot(fpr_rf, tpr_rf, color='pink', linestyle='dashdot', label=f'Random Forest (AUC = {auc_rf:.2f})')

# Random guess line
plt.plot([0, 1], [0, 1], color='black', linestyle='dotted', label='Random Guess')

# Labels and Title
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve Comparison: All Classification Models")
plt.grid(True)
plt.legend(loc='best') 
plt.show()

### The best model is the Random Forest.

## AFTER SMOTE

In [ ]:
Y.value_counts()

In [ ]:
#Apply SMOTE
smote = SMOTE(sampling_strategy='minority')
X, Y = smote.fit_resample(X, Y)

In [ ]:
Y.value_counts()

In [ ]:
X.shape

**After SMOTE, I have 74114 rows and 16 columns**

In [ ]:
#Apply SMOTE for Trees data set
smote = SMOTE(sampling_strategy='minority')
X_t, Y_t = smote.fit_resample(X_t, Y_t)

In [ ]:
Y_t.value_counts()

In [ ]:
X_t.shape

**After SMOTE, I have 74114 rows and 13 columns for Trees Data set**

## Feature Scaling

In [ ]:
# split data for Linear model
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

ro_scaler = RobustScaler()
X_train_scaled = ro_scaler.fit_transform(X_train)
X_test_scaled = ro_scaler.transform(X_test)

X_train = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test = pd.DataFrame(X_test_scaled, columns=X.columns)

X_train.shape

In [ ]:
# split data for trees model

X_t_train, X_t_test, Y_t_train, Y_t_test = train_test_split(X_t, Y_t, test_size=0.2, random_state=42)

## ML modeling

### Logistic Regression

In [ ]:
# Logistic Regression
logreg = LogisticRegression(random_state=16, max_iter=2000)
logreg.fit(X_train, Y_train)

# probablity for auc-roc cure
lg_probs = logreg.predict_proba(X_test)

In [ ]:
y_pred_train = logreg.predict(X_train)
y_pred = logreg.predict(X_test)

conf_matrix = confusion_matrix(Y_test, y_pred)
conf_matrix

- True Negatives (TN) = 5825 <br>
The model correctly predicted 5825 people as earning ≤50K.
- False Positives (FP) = 1640<br>
The model incorrectly predicted 1640 people as earning >50K, when they actually earn ≤50K (Type I error).
- False Negatives (FN) = 1384<br>
The model incorrectly predicted 1384 people as earning ≤50K, when they actually earn >50K (Type II error).
- True Positives (TP) = 5974<br>
The model correctly predicted 5974 people as earning >50K.

In [ ]:
# probablity

print(f"Train Accuracy score for Logistic Regression: {accuracy_score(Y_train, y_pred_train):.2f}")
print(f"Test Accuracy score for Logistic Regression: {accuracy_score(Y_test, y_pred):.2f}")


target_names = ['<=50K', '>50K']
print(classification_report(Y_test, y_pred, target_names = target_names))

- Observations
	1.	Balanced Performance → The model performs similarly across both classes, meaning it does not strongly favor one over the other.
	2.	Recall for >50K is slightly higher than ≤50K (0.82 vs. 0.78) → The model is slightly better at capturing high-income individuals.
	3.	Precision and Recall are close, leading to a well-balanced F1-score → No extreme trade-off between false positives and false negatives.

- Final Verdict
    1. The model performs well with an overall F1-score of 0.80, indicating a reliable classification performance.
    2. Some improvement can be made in recall for ≤50K (0.78), which means the model might be missing some low-income individuals.
	

### KNN

In [ ]:
error_rates = []
k_values = range(1, 101)
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, Y_train)
    Y_knn_pred = knn.predict(X_test)
    error = 1 - accuracy_score(Y_test, Y_knn_pred)
    error_rates.append(error)

In [ ]:
#Plot the elbow curve to find the optimal value of k
plt.figure(figsize=(20, 5))
plt.plot(k_values, error_rates, marker='o', linestyle='dashed', color='b', markersize=8)
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Error Rate')
plt.title('Elbow Method to Find Optimal k')
plt.xticks(np.arange(1, 101))
plt.grid(True)
plt.show()

**The error rate is the least for k = 3.**

In [ ]:
#Performing KNN with optimal value of k
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, Y_train)

Y_knn_probs = knn.predict_proba(X_test)
Y_knn_probs

y_pred_train = knn.predict(X_train)
Y_knn_pred = knn.predict(X_test)

conf_matrix = confusion_matrix(Y_test, Y_knn_pred)
conf_matrix

- True Negatives (TN) = 6300 <br>
The model correctly predicted 6300 people as earning ≤50K.
- False Positives (FP) = 1165<br>
The model incorrectly predicted 1165 people as earning >50K, when they actually earn ≤50K (Type I error).
- False Negatives (FN) = 1076<br>
The model incorrectly predicted 1076 people as earning ≤50K, when they actually earn >50K (Type II error).
- True Positives (TP) = 6282<br>
The model correctly predicted 6282 people as earning >50K.

In [ ]:
print(f"Train Accuracy score for KNN: {accuracy_score(Y_train, y_pred_train):.2f}")
print(f"Test Accuracy score for KNN: {accuracy_score(Y_test, Y_knn_pred):.2f}")

target_names = ['≤50K', '>50K']
print(classification_report(Y_test, Y_knn_pred, target_names = target_names))

- Observations
	1.	Balanced Classification → The model performs equally well on both classes, with no significant bias toward one.
	2.	Potential Overfitting → The 7% accuracy gap suggests that the model may be slightly overfitting. Reducing the number of neighbors (k) or applying regularization techniques might improve generalization.
	3.	Strong Overall Performance → With an F1-score of 0.85, the model is reliable and effective for classification.

- Final Verdict
	1. The KNN model performs well, with balanced precision, recall, and F1-scores across both classes.
    2. There is some overfitting, but it is not severe. Fine-tuning hyperparameters (e.g., increasing k) could help improve test performance.
	

In [ ]:
cv_scores = cross_val_score(knn, X_train, Y_train, cv=5)
print(f"Cross-validation mean accuracy: {cv_scores.mean():.2f}" )

**I initially thought this model was overfitting, but after comparing the test accuracy with the cross-validation results, I found them to be same, so I concluded that there is no issue.**

### Decision Tree

In [ ]:
#Find the best value for max_depth in decision tree
scores = []
depths = range(1, 21)
for d in depths:
    score = cross_val_score(DecisionTreeClassifier(criterion= 'entropy', max_depth=d), X_t_train, Y_t_train, cv=5)
    avg_score = mean(score)
    scores.append(avg_score)

best_depth = depths[np.argmax(scores)]
print(f"Best max_depth: {best_depth}")

In [ ]:
dt_clr_opt = DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=12)
dt_clr_opt.fit(X_t_train, Y_t_train)

#predict
y_pred_train = dt_clr_opt.predict(X_t_train)
y_pred_test = dt_clr_opt.predict(X_t_test)

conf_matrix = confusion_matrix(Y_t_test, y_pred_test)
conf_matrix

- True Negatives (TN) = 6136<br>
The model correctly predicted 6136 people as earning ≤50K.
- False Positives (FP) = 1329<br>
The model incorrectly predicted 1329 people as earning >50K when they actually earn ≤50K (Type I error).
- False Negatives (FN) = 995<br>
The model incorrectly predicted 995 people as earning ≤50K when they actually earn >50K (Type II error).
- True Positives (TP) = 6363<br>
The model correctly predicted 6363 people as earning >50K.

In [ ]:
print(f"Train Accuracy:, {accuracy_score(Y_t_train, y_pred_train):.2f}")
print(f"Test Accuracy:, {accuracy_score(Y_t_test, y_pred_test):.2f}")
target_names = ['≤50K', '>50K']
print("\nClassification Report:\n", classification_report(Y_t_test, y_pred_test, target_names=target_names))

- Observations
	1.	Balanced Performance → The model performs similarly across both classes, meaning it does not strongly favor one over the other.
	2.	Recall for >50K is slightly higher than ≤50K (0.86 vs. 0.83) → The model is slightly better at capturing high-income individuals.
	3.	Precision and Recall are close, leading to a well-balanced F1-score → No extreme trade-off between false positives and false negatives.

- Final Verdict
    1. The model performs well with an overall F1-score of 0.84, indicating a reliable classification performance.
    2. Some improvement can be made in recall for ≤50K (0.83), which means the model might be missing some low-income individuals.
	

In [ ]:
cv_scores = cross_val_score(dt_clr_opt, X_t_train, Y_t_train, cv=5)
print(f"Cross-validation mean accuracy: {cv_scores.mean():.2f}" )

**I initially thought this model was overfitting, but after comparing the test accuracy with the cross-validation results, I found them to be same, so I concluded that there is no issue.**

### Plot the decision tree

In [ ]:
# Export the decision tree to DOT format
dot_data = StringIO()

export_graphviz(
    dt_clr_opt,
    out_file=dot_data, 
    filled=True,  
    rounded=True,  
    special_characters=True,  
    feature_names=X_t_train.columns, 
    class_names=['Low income','High income']
)

# Generate graph
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png("decision_tree_SMOTE.png")  # Save as PNG

# Display the tree
Image(filename="decision_tree_SMOTE.png")

### SVM

In [ ]:
from sklearn.utils import resample

X_train_sample, Y_train_sample = resample(X_train, Y_train, n_samples=5000, random_state=42)  # 5000개만 사용

grid_search.fit(X_train_sample, Y_train_sample)

svm = SVC() #Model building

param_grid = {"C": [10, 100, 200, 300]}

# GridSearchCV
grid_search = GridSearchCV(svm, param_grid, cv=3, scoring="accuracy", n_jobs=-1)
grid_search.fit(X_train_sample, Y_train_sample)

print(f"Best C: {grid_search.best_params_['C']}")

In [ ]:
svm_300 = SVC(random_state=42, C=300, probability=True)
svm_300.fit(X_train, Y_train)

Y_svm_probs = svm_300.predict_proba(X_test)

y_pred_train = svm_300.predict(X_train)
y_pred_test = svm_300.predict(X_test)

conf_matrix = confusion_matrix(Y_test, y_pred_test)
conf_matrix

- True Negatives (TN) = 5521
The model correctly predicted 5521 people as earning ≤50K.
- False Positives (FP) = 1944
The model incorrectly predicted 1944 people as earning >50K, when they actually earn ≤50K (Type I error).
- False Negatives (FN) = 1094
The model incorrectly predicted 1094 people as earning ≤50K, when they actually earn >50K (Type II error).
- True Positives (TP) = 6264
The model correctly predicted 6264 people as earning >50K.

In [ ]:
# train and test accuracy
print(f"Train Accuracy:, {accuracy_score(Y_train, y_pred_train):.2f}")
print(f"Test Accuracy:, {accuracy_score(Y_test, y_pred_test):.2f}")

target_names = ['≤50K', '>50K']
print(classification_report(Y_test, y_pred_test, target_names = target_names))


- Key Observations
	1.	No Overfitting → Since train and test accuracy are the same (80%), the model generalizes well.
	2.	Class Imbalance Impact
    	-	The model is slightly biased toward the >50K class since it has a higher recall (0.85 vs. 0.74).
    	-	This means the model misses more actual ≤50K cases (higher False Negatives) than it does for >50K cases.
	3.	Better Recall for >50K Class → The model is more effective at capturing >50K earners (85%) than ≤50K earners (74%).
	4.	Trade-off Between Precision & Recall
    	-	The ≤50K class has higher precision (0.83) but lower recall (0.74), meaning it is good at confirming ≤50K cases but tends to miss some.
    	-	The >50K class has lower precision (0.76) but higher recall (0.85), meaning it captures most >50K cases but with slightly more False Positives.
- Final Verdict
    1. Balanced model but slightly favors the >50K class
    2. Good generalization (no overfitting)
    3. Might need adjustment to improve recall for ≤50K class 

### Random Forest

In [ ]:
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)

param_grid = {
    "n_estimators": [50, 100, 200],  # trees
    "max_depth": [10, 20, None],  
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4] 
}
# for better one
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring="accuracy", n_jobs=-1, verbose=2)
grid_search.fit(X_t_train, Y_t_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

In [ ]:
# the best model
best_rf = grid_search.best_estimator_

y_pred_train = best_rf.predict(X_t_train)
y_pred_test = best_rf.predict(X_t_test)

print(f"Train Accuracy: {accuracy_score(Y_t_train, y_pred_train):.2f}")
print(f"Test Accuracy: {accuracy_score(Y_t_test, y_pred_test):.2f}")
print()
# Confusion Matrix
conf_matrix = confusion_matrix(Y_t_test, y_pred_test)
print("Confusion Matrix:\n", conf_matrix)
print()
# Classification Report
print("Classification Report:\n", classification_report(Y_t_test, y_pred_test, target_names=["≤50K", ">50K"]))

- True Negatives (TN) = 6254
The model correctly predicted 6254 people as earning ≤50K.

- False Positives (FP) = 1211 (Type I error)
The model incorrectly predicted 1211 people as earning >50K when they actually earn ≤50K.

- False Negatives (FN) = 737 (Type II error)
The model incorrectly predicted 737 people as earning ≤50K when they actually earn >50K.

- True Positives (TP) = 6621
The model correctly predicted 6621 people as earning >50K.

- Key Observations
    1. Potential Overfitting -> the 7% accuarcy gap suggests that the model may be slightly overfitting.
    2. Strong Overall Performance -> with an F1-score of 0.87, the model is reliable and effective for classification.
    3. More False Positives (1211) than False Negatives (737) → The model overestimates high-income individuals more often than it misses them.

In [ ]:
cv_scores = cross_val_score(best_rf, X_t_train, Y_t_train, cv=5)
print("Cross-validation mean accuracy:", cv_scores.mean())

**I initially thought this model was overfitting, but after comparing the test accuracy with the cross-validation results, I found them to be similar, so I concluded that there is no issue.**

## AUC-ROC curve

In [ ]:
# Graph for only positive
Y_lr_probs = lg_probs[:, 1]
Y_knn_probs = Y_knn_probs[:, 1]
Y_dt_probs = dt_clr_opt.predict_proba(X_t_test)[:, 1]
Y_svm_probs = Y_svm_probs[:, 1]
Y_rf_probs = best_rf.predict_proba(X_t_test)[:, 1]

# Compute ROC curve and AUC for all the models
# Logistic Regression
fpr_lr, tpr_lr, _ = roc_curve(Y_test, Y_lr_probs)
auc_lr = auc(fpr_lr, tpr_lr)
# KNN
fpr_knn, tpr_knn, _ = roc_curve(Y_test, Y_knn_probs)
auc_knn = auc(fpr_knn, tpr_knn)
# Decision Trees
fpr_dt, tpr_dt, _ = roc_curve(Y_t_test, Y_dt_probs)
auc_dt = auc(fpr_dt, tpr_dt)
# SVM
fpr_svm, tpr_svm, _ = roc_curve(Y_test, Y_svm_probs)
auc_svm = auc(fpr_svm, tpr_svm)
# Random Forest
fpr_rf, tpr_rf, _ = roc_curve(Y_t_test, Y_rf_probs)
auc_rf = auc(fpr_rf, tpr_rf)

# Plot both ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr_lr, tpr_lr, color='blue', label=f'Logistic Regression (AUC = {auc_lr:.2f})')
plt.plot(fpr_knn, tpr_knn, color='red', linestyle='dashed', label=f'KNN (k=12) (AUC = {auc_knn:.2f})')
plt.plot(fpr_dt, tpr_dt, color='green', linestyle='dashdot', label=f'Decision Tree (AUC = {auc_dt:.2f})')
plt.plot(fpr_svm, tpr_svm, color='orange', linestyle='dashed', label=f'SVM (AUC = {auc_svm:.2f})')
plt.plot(fpr_rf, tpr_rf, color='pink', linestyle='dashdot', label=f'Random Forest (AUC = {auc_rf:.2f})')

# Random guess line
plt.plot([0, 1], [0, 1], color='black', linestyle='dotted', label='Random Guess')

# Labels and Title
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve Comparison: All Classification Models")
plt.grid(True)
plt.legend(loc='best') 
plt.show()

### Conclusion: Model Performance Before and After Addressing Class Imbalance

**Before Handling Class Imbalance**
- Random Forest (AUC = 0.90) performed the best, followed closely by KNN (AUC = 0.89) and Decision Tree (AUC = 0.88).
- SVM (AUC = 0.83) had the lowest AUC, suggesting it struggled more compared to the other models.

**After Applying SMOTE (Handling Class Imbalance)**
- Random Forest showed the highest improvement, increasing from AUC 0.90 to 0.95, indicating it benefited significantly from SMOTE.
- KNN and Decision Tree also improved to AUC 0.91, making them strong contenders.
- Logistic Regression and SVM improved from 0.84 → 0.88 and 0.83 → 0.88, respectively, showing that balancing the dataset positively impacted their performance.

**Key Observations & Interpretation**<br>
1. All models improved after handling class imbalance, suggesting that SMOTE helped address the bias toward the majority class.<br>
2. Random Forest consistently performed the best both before and after SMOTE, making it the most robust model in this scenario.<br>
3. Decision Tree and KNN saw notable improvements, indicating they were sensitive to class imbalance and benefited from synthetic data augmentation.<br>
4. SVM and Logistic Regression had the lowest AUC initially but showed considerable improvement after SMOTE, meaning they required balanced data to perform well.

**Final Verdict**
- Best Model Before Class Imbalance → Random Forest (AUC = 0.90)
- Best Model After SMOTE → Random Forest (AUC = 0.95)
- Most Improved Models → Decision Tree & KNN (AUC increased to 0.91)
- Overall Recommendation → Random Forest remains the best choice, but Decision Tree and KNN are also strong contenders after handling class imbalance.